# Your Title Information Goes Here
Solo Jacket Project

## Setup

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`


   Resolving package versions...


  No Changes to `~/Documents/GitHub/CHEME-5660-Project-Template-F23/Project.toml`
    Updating `~/Documents/GitHub/CHEME-5660-Project-Template-F23/Manifest.toml`
  [79e6a3ab] ↑ Adapt v3.7.1 ⇒ v3.7.2
  [7a1cc6ca] ↑ FFTW v1.7.1 ⇒ v1.7.2
  [1a297f60] ↑ FillArrays v1.9.2 ⇒ v1.9.3
  [5ab0869b] ↑ KernelDensity v0.6.7 ⇒ v0.6.8
  [5c8ed15e] ↑ LinearOperators v2.5.2 ⇒ v2.6.0
  [b8a86587] ↑ NearestNeighbors v0.4.13 ⇒ v0.4.15
  [90014a1f] ↑ PDMats v0.11.30 ⇒ v0.11.31
  [90137ffa] ↑ StaticArrays v1.7.0 ⇒ v1.8.0
  [aacddb02] ↑ JpegTurbo_jll v2.1.91+0 ⇒ v3.0.1+0
  [856f044c] ↑ MKL_jll v2023.2.0+0 ⇒ v2024.0.0+0
  [b53b4c65] ↑ libpng_jll v1.6.38+1 ⇒ v1.6.40+0


Precompiling 

project...


...

  ✓ VLQuantitativeFinancePackage
  7 dependencies successfully precompiled in 12 seconds. 249 already precompiled.
  Activating project at `~/Documents/GitHub/CHEME-5660-Project-Template-F23`


  No Changes to `~/Documents/GitHub/CHEME-5660-Project-Template-F23/Project.toml`
  No Changes to `~/Documents/GitHub/CHEME-5660-Project-Template-F23/Manifest.toml`


    Updating registry at `~/.julia/registries/General.toml`


    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`


    Updating `~/Documents/GitHub/CHEME-5660-Project-Template-F23/Project.toml`
  [31c24e10] ↑ Distributions v0.25.103 ⇒ v0.25.104
    Updating `~/Documents/GitHub/CHEME-5660-Project-Template-F23/Manifest.toml`
  [31c24e10] ↑ Distributions v0.25.103 ⇒ v0.25.104


## Prerequisites: Load historical dataset, compute expected returns and get 2023 `SPY` data
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) since `01-03-2018` until `12-01-2023`, along with data for a few exchange traded funds and volatility products during that time. 

In [8]:
original_dataset = load(joinpath(_PATH_TO_DATA, 
        "SP500-Daily-OHLC-1-3-2018-to-12-01-2023.jld2")) |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [9]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [10]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it in the `all_tickers` array:

In [11]:
all_tickers = keys(dataset) |> collect |> sort;
K = length(all_tickers);

### Get the 2023 `SPY` data

In [12]:
startdate = Date(2023,01,03);
SPY_dataset = dataset["SPY"];
SPY_df = filter(:timestamp => x-> x >= startdate, SPY_dataset);

### Compute the expected return for all firms in the dataset
The expected return $\mathbb{E}(r_{i})$ and covariance matrix $\Sigma$ will be used in our calculations, so we'll provide values for both of these items for the entire data set (all `N = 459` tickers), and then you can pick out which tickers you are interested in. 

* First, we compute the expected (annualized) log return by passing the `dataset` and the entire list of firms we have in the dataset (held in the $N\times{1}$ `all_array` array) to the `log_return_matrix(...)` method. The result is stored in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log return values. Each row of `all_firms_return_matrix` corresponds to a time-value, while each column corresponds to a firm:

In [13]:
all_firms_return_matrix = log_return_matrix(dataset, all_tickers, 
    Δt = (1.0/252.0), risk_free_rate = 0.0);

## Your project starts here ....

In [14]:
## Fill me in
#Lets get started 
μ = mean(all_firms_return_matrix, dims=1) |> vec;

Now before doing anything else lets break down what we need to achieve:
1. Identify Capital and Operating Costs:
Capital Expenses:
Building and Equipment: $5 million
Annual Operating Expenses (for each clinic):
Staffing (2 doctors, 3 Physician Assistants): 5 * (2 + 3) * Median Local Annual Wage
Supply and Utility Costs: $10,000 * 12
Total Operating Expenses per Clinic: Sum of Staffing and Supply/Utility Costs
Total Annual Operating Expenses (16 clinics): 16 * Total Operating Expenses per Clinic
2. Structure the Endowment:
Determine the annual endowment amount required to cover operating expenses and 20% reinvestment payment.
Ensure the face value of the endowment covers the initial capital expense ($5 million).
3. Quantify the Initial Grant Size and Develop an Investment Portfolio:
Use historical risky asset pricing data (2018-2022) and risk-free auction data (2022) to define an investment portfolio.
Evaluate portfolio performance based on 2023 data.
If the portfolio doesn't meet obligations, consider reallocating assets.


Average salary for physician assistant: 57,719,800 UGX or 15,228.39 United States Dollar #https://worldsalaries.com/average-physician-assistant-salary-in-kampala/uganda/
Average salary for doctors: 89,281,500 UGX or 23,555.41 United States Dollar #https://worldsalaries.com/average-doctor-salary-in-kampala/uganda/


In [26]:
# Constants
discount_rate = 0.05
number_of_clinics = 15
monthly_utility_costs_usd = 10000
initial_building_and_equipment_expense_usd = 5000000

# Function to calculate present value using discounting
function present_value(future_value, discount_rate, years)
    return future_value / (1 + discount_rate)^years
end

# Operating Expenses Calculation
pa_salary_usd = 15228.39 * 3  # 3 Physician Assistants per clinic
doctor_salary_usd = 23555.41 * 2  # 2 Doctors per clinic

# Total Staffing Cost per Clinic
total_staffing_cost_per_clinic_usd = pa_salary_usd + doctor_salary_usd

# Monthly Supply and Utility Costs per Clinic
total_supply_utility_costs_per_clinic_usd = monthly_utility_costs_usd * 12

# Total Operating Expenses per Clinic
total_operating_expenses_per_clinic_usd = total_staffing_cost_per_clinic_usd + total_supply_utility_costs_per_clinic_usd

# Total Annual Operating Expenses for All Clinics
total_annual_operating_expenses_usd = total_operating_expenses_per_clinic_usd * number_of_clinics

# Present Value of Total Annual Operating Expenses over 10 years (discounted)
present_value_total_annual_expenses_usd = present_value(total_annual_operating_expenses_usd, discount_rate, 10)

# Present Value of Total Capital Expenses (building and equipment) over 10 years (discounted)
present_value_building_and_equipment_expense_usd = present_value(initial_building_and_equipment_expense_usd, discount_rate, 10)

using Printf



println("Total Operating Expenses per Clinic (USD): $(@sprintf("%.2f", total_operating_expenses_per_clinic_usd))")
println("Total Annual Operating Expenses for All Clinics (USD): $(@sprintf("%.2f", total_annual_operating_expenses_usd))")
println("Present Value of Total Annual Operating Expenses over 10 years (USD): $(@sprintf("%.2f", present_value_total_annual_expenses_usd))")
println("Present Value of Total Capital Expenses (building and equipment) over 10 years (USD): $(@sprintf("%.2f", present_value_building_and_equipment_expense_usd))")


Total Operating Expenses per Clinic (USD): 212795.99
Total Annual Operating Expenses for All Clinics (USD): 3191939.85
Present Value of Total Annual Operating Expenses over 10 years (USD): 1959574.18
Present Value of Total Capital Expenses (building and equipment) over 10 years (USD): 3069566.27


## Disclaimer and Risks
__This content is offered solely for training and  informational purposes__. No offer or solicitation to buy or sell securities or derivative products, or any investment or trading advice or strategy,  is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on your evaluation of your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.